In [1]:
import pandas as pd

df_2016 = pd.read_csv("data/2016_important/whole_data.csv", low_memory = False)

df_2016["Period"] = df_2016["Period"].apply(lambda val : val.strip())

df_2016["Time_begin"] = pd.to_datetime(df_2016["Time_begin"], format="%H:%M:%S.%f")
df_2016["Time_end"] = pd.to_datetime(df_2016["Time_end"], format="%H:%M:%S.%f")
df_2016["Duration"] = pd.to_datetime(df_2016["Duration"], format="%H:%M:%S.%f")

df_2016["Time_begin"] = df_2016["Time_begin"].dt.round(freq = '500L', nonexistent="shift_forward")
df_2016["Time_end"] = df_2016["Time_end"].dt.round(freq = '500L',  nonexistent="shift_backward")

check_series = df_2016["Time_begin"].isna()
df_2016 = df_2016[~check_series]

df_2016["Duration"] = df_2016["Time_end"] - df_2016["Time_begin"]
df_2016["Duration"] = df_2016["Duration"].dt.total_seconds()

df_2016["NVB"] = df_2016[["Gaze_Worksheet_Tutor","Gaze_Worksheet_Tutee","Gaze_Elsewhere_Tutor","Gaze_Elsewhere_Tutee","Gaze_Partner_Tutor","Gaze_Partner_Tutee","Smile_Tutor","Smile_Tutee"]].replace("x",1).values.tolist()

df_2016["Timestamps"] = df_2016[["Time_begin", "Time_end"]].apply(lambda row : pd.date_range(row["Time_begin"], row["Time_end"], freq = "500L"), axis = 1)

In [117]:
df_lookup = pd.DataFrame(columns = ["Dyad", "Session", "Period", "Time", "NVB"])

df_time_begin = df_2016[["Dyad", "Session", "Time_begin"]].groupby(by = ["Dyad", 'Session']).min()
df_time_end = df_2016[["Dyad", "Session", "Time_end"]].groupby(by = ["Dyad", 'Session']).max()

dict_temp_df = {}

nvb_list = ["Gaze_Worksheet_Tutor","Gaze_Worksheet_Tutee","Gaze_Elsewhere_Tutor","Gaze_Elsewhere_Tutee","Gaze_Partner_Tutor","Gaze_Partner_Tutee","Smile_Tutor","Smile_Tutee"]

for dyad_ in df_2016["Dyad"].unique():
    df_dyad_ = df_2016[df_2016["Dyad"] == dyad_]
    for session_ in df_dyad_["Session"].unique():
        df_dyad_session_ = df_dyad_[df_dyad_["Session"] == session_]
        time_begin = df_time_begin.loc[dyad_, session_].values[0]
        time_end = df_time_end.loc[dyad_, session_].values[0]
        series_range = pd.date_range(time_begin, time_end, freq="500L")

        temp_df_ = pd.DataFrame(columns = ["Dyad", "Session", "Time", "NVB"])
        temp_df_["Time"] = series_range
        temp_df_["Dyad"] = dyad_
        temp_df_["Session"] = session_

        for ts_ in series_range:
            print(df_dyad_session_[(df_dyad_session_["Time_begin"] < ts_)&(df_dyad_session_["Time_end"] > ts_)]["NVB"].sum(axis=1))
            temp_df_[temp_df_["Time"] == ts_]["NVB"] = df_dyad_session_[(df_dyad_session_["Time_begin"] < ts_)&(df_dyad_session_["Time_end"] > ts_)]["NVB"].sum()


        dict_temp_df["d"+str(dyad_) + "s" + str(session_)] = temp_df_

for df_key_ in list(dict_temp_df.keys()):
    df_lookup = pd.concat([df_lookup, dict_temp_df[df_key_]])

df_lookup = df_lookup.drop(columns = ["Period"])

ValueError: No axis named 1 for object type Series

In [3]:
import pandas as pd

df_2016 = pd.read_csv("data/2016_important/2016_final.csv", low_memory = False)

df_2016["Duration_s"] = pd.to_timedelta(df_2016["Duration"]).dt.total_seconds()
df_2016["Duration_s"] = df_2016["Duration_s"].apply(lambda val : min(val, 12.0))
df_2016 = df_2016.sort_values("Duration_s")

df_2016 = df_2016.dropna(subset=['P1', 'P2'], how='all')

In [11]:
import pandas as pd
df_2016 = pd.read_csv("data/2016_important/2016_aus_r.csv", low_memory = False)

In [12]:
def round5(number):
    return (round(number * 2) / 2)

df_2016[" timestamp"] = df_2016[" timestamp"].apply(round5)


In [13]:
df_2016

,dyad,session,frame,face_id,timestamp,confidence,success,AU01_r,AU02_r,AU04_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,10,1,1,0,0.0,0.00,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10,1,2,0,0.0,0.00,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,1,3,0,0.0,0.00,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10,1,4,0,0.0,0.00,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,1,5,0,0.0,0.00,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6610780,9,2,103673,0,3459.0,0.98,1,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6610781,9,2,103674,0,3459.0,0.98,1,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6610782,9,2,103675,0,3459.5,0.98,1,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6610783,9,2,103676,0,3459.5,0.98,1,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
df_20162 = df_2016.groupby(['dyad', 'session', ' timestamp']).mean()

In [19]:
df_2016 = df_2016.groupby(['dyad', 'session', ' timestamp']).first()

In [24]:
df_2016[[' confidence', ' AU01_r',' AU02_r',' AU04_r',' AU05_r',' AU06_r',' AU07_r',' AU09_r', ' AU10_r', ' AU12_r', ' AU14_r', ' AU15_r', ' AU17_r', ' AU20_r', ' AU23_r', ' AU25_r', ' AU26_r', ' AU45_r']] = df_20162[[' confidence', ' AU01_r',' AU02_r',' AU04_r',' AU05_r',' AU06_r',' AU07_r',' AU09_r', ' AU10_r', ' AU12_r', ' AU14_r', ' AU15_r', ' AU17_r', ' AU20_r', ' AU23_r', ' AU25_r', ' AU26_r', ' AU45_r']]

In [26]:
reset_df = df_2016.reset_index().drop(columns = [' AU01_c',' AU02_c',' AU04_c',' AU05_c',' AU06_c',' AU07_c',' AU09_c',' AU10_c',' AU12_c',' AU14_c',' AU15_c',' AU17_c',' AU20_c',' AU23_c',' AU25_c',' AU26_c',' AU28_c',' AU45_c'] )

In [35]:
reset_df = reset_df.drop(columns=' face_id')

In [38]:
final_df = reset_df.rename(columns = {' AU01_r':'AU01_r', ' AU02_r':'AU02_r', ' AU04_r':'AU04_r', ' AU05_r':'AU05_r',
       ' AU06_r':'AU06_r', ' AU07_r':'AU07_r', ' AU09_r':'AU09_r', ' AU10_r':'AU10_r', ' AU12_r':'AU12_r', ' AU14_r':'AU14_r',
       ' AU15_r':'AU15_r', ' AU17_r':'AU17_r', ' AU20_r':'AU20_r', ' AU23_r':'AU23_r', ' AU25_r':' AU25_r', ' AU26_r':' AU26_r',
       ' AU45_r':'AU45_r'})

In [40]:
final_df

,dyad,session,timestamp,confidence,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r
0,1,1,0.0,0.584286,0.152857,0.000000,0.445714,0.178571,0.645714,1.350000,...,0.101429,0.521429,0.538571,0.784286,1.372857,0.238571,1.522857,0.530000,0.455714,0.338571
1,1,1,0.5,0.856154,0.578462,0.698462,0.522308,0.003077,0.741538,1.473846,...,0.673077,1.153846,1.084615,0.968462,1.065385,1.184615,0.947692,1.273846,1.219231,0.139231
2,1,1,1.0,0.724615,0.831538,0.578462,0.303077,0.011538,0.603077,1.858462,...,0.293846,1.136923,1.072308,0.332308,0.879231,0.823077,0.646154,1.585385,1.005385,0.050769
3,1,1,1.5,0.847857,1.556429,1.205714,0.176429,0.681429,0.847857,1.312143,...,0.402857,0.857857,0.407143,0.187143,0.640000,0.127143,0.637143,0.722857,0.837143,0.101429
4,1,1,2.0,0.895385,1.340000,1.415385,0.292308,0.723846,0.884615,0.924615,...,0.053077,0.724615,0.743077,0.063846,0.163846,0.080000,0.947692,0.980000,0.127692,0.111538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247921,21,1,3495.0,0.951667,0.117333,0.095000,0.544667,0.342000,0.742333,0.503333,...,0.295000,0.986333,0.955333,0.001000,0.482667,0.131000,1.333333,0.829333,0.235667,0.000000
247922,21,1,3495.5,0.955000,0.003667,0.015000,0.334333,0.256333,0.546000,0.616333,...,0.093667,1.008667,0.750000,0.003333,0.859333,0.170000,0.991667,0.549667,0.688333,0.068000
247923,21,1,3496.0,0.718667,0.227000,0.327667,0.278000,0.091333,1.223667,1.364333,...,0.649000,1.379000,0.861667,0.249333,0.629000,0.732000,0.538333,1.626000,1.026667,0.145333
247924,21,1,3496.5,0.950000,0.000000,0.278333,0.291667,0.394333,1.153333,1.385667,...,1.009667,1.194667,0.879333,0.052333,1.026333,0.889000,1.407667,1.396000,1.328000,0.027000


In [41]:
test_df = pd.read_csv("data/2016_raw/action_units/2016_aus_r_500ms.csv")

In [1]:
import pandas as pd

df_sm_g = pd.read_csv("data/2016_important/2016_gaze_smile.csv")

C:\Users\Alafate\AppData\Local\Temp\ipykernel_10148\722606648.py:3: DtypeWarning: Columns (8,9,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sm_g = pd.read_csv("data/2016_important/2016_gaze_smile.csv")


In [12]:
df_sm_g = df_sm_g[['Dyad','Session','Period','Time_begin','Time_end','Duration','P1','P2','Gaze_Worksheet_Tutor','Gaze_Worksheet_Tutee','Gaze_Elsewhere_Tutor','Gaze_Elsewhere_Tutee','Gaze_Partner_Tutor','Gaze_Partner_Tutee','Smile_Tutor','Smile_Tutee']]

df_sm_g["participant"] = df_sm_g[["P1", "P2"]].apply(lambda row : int(len(str(row["P1"])) > 3)*"1" + int(len(str(row["P2"]))> 3)*"2", axis = 1)

In [13]:
df_sm_g

,Dyad,Session,Period,Time_begin,Time_end,Duration,P1,P2,Gaze_Worksheet_Tutor,Gaze_Worksheet_Tutee,Gaze_Elsewhere_Tutor,Gaze_Elsewhere_Tutee,Gaze_Partner_Tutor,Gaze_Partner_Tutee,Smile_Tutor,Smile_Tutee,participant
0,3,1,NaN,00:00:00.0,00:00:04.3,00:00:04.3,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,
1,3,1,NaN,00:00:00.0,00:00:07.0,00:00:07.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,3,1,NaN,00:00:00.0,00:00:07.0,00:00:07.0,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,
3,3,1,NaN,00:00:00.0,00:00:05.6,00:00:05.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,
4,3,1,S1,00:00:00.1,00:05:12.0,00:05:11.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69681,18,2,T2,00:59:31.4,00:59:31.7,00:00:00.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,
69682,18,2,T2,00:59:31.4,00:59:31.6,00:00:00.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,
69683,18,2,T2,00:59:34.8,00:59:35.0,00:00:00.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,
69684,18,2,T2,00:59:35.4,00:59:36.1,00:00:00.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,


In [2]:
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
a = tokenizer.encode_plus(
            "<s> hello this is the house of the final excipient of lighting stars </s>",
            None,
            add_special_tokens=True,
            max_length=50,
            padding="max_length",
            return_token_type_ids=True,
            return_tensors = 'pt'
        )["input_ids"][0]
tokenizer.convert_ids_to_tokens(a)

['<s>',
 '<s>',
 'Ġhello',
 'Ġthis',
 'Ġis',
 'Ġthe',
 'Ġhouse',
 'Ġof',
 'Ġthe',
 'Ġfinal',
 'Ġexc',
 'ipient',
 'Ġof',
 'Ġlighting',
 'Ġstars',
 'Ġ',
 '</s>',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [14]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<pad>', '<mask>']

In [15]:
a

tensor([    0,     0, 42891,    42,    16,     5, 14188, 14188,     9,     5,
          821,  6621,   821,  6621,     2,     2,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1])

In [16]:
tokenizer.convert_ids_to_tokens(a)

['<s>',
 '<s>',
 'hello',
 'Ġthis',
 'Ġis',
 'Ġthe',
 'Ġzoo',
 'Ġzoo',
 'Ġof',
 'Ġthe',
 'Ġg',
 'aa',
 'Ġg',
 'aa',
 '</s>',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']